In [1]:
#where to load the dataset labels
test_path = 'data/images_scraped/'

In [2]:
#where to load the net to use
SAVE_PATH = 'trained/scheduler_resnet/new_res18_25_best.pth'

using_res = True

### Imports

In [ ]:
import torch, torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize, ToPILImage, CenterCrop, Normalize, Compose
from torchvision.transforms.functional import to_grayscale, to_tensor, rotate, hflip
import matplotlib.pyplot as plt

import os
import random
import pandas as pd
from torchvision.io import read_image

from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from sklearn.metrics import classification_report, confusion_matrix

from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
import seaborn as sn
import numpy as np

import io
from PIL import Image
from PIL.features import pilinfo

from copy import deepcopy


import traceback
import warnings
warnings.filterwarnings("error")

In [ ]:
only_dirs = [ name for name in os.listdir(test_path) if 
                 os.path.isdir(os.path.join(test_path, name)) ]

diz = {}
nocl=0 #num of classes
for d in only_dirs:
    diz[nocl] = d
    nocl+=1
    
print(diz.values())

In [ ]:
#where the images are located and their class
test_path2 = 'images/hyundai-conv1/'
SINGLE_CLASS = 'hyundai'

LABEL_INDEX = list(diz.values()).index(SINGLE_CLASS)

### Useful functions to retrive the data

In [ ]:
# create a annotations_file.csv (index -> img (name), label (int))

def getLabelList(path):
    only_dirs = [ name for name in os.listdir(path) if 
                 os.path.isdir(os.path.join(path, name)) ]

    ret = {}
    index = 0
    
    for d in only_dirs:
        new_path = path + d
        label = only_dirs.index(d)
        # print(len(os.listdir(new_path)))
        for img in [ name for name in os.listdir(new_path) ]:
            ret[index] = [img, label]
            index += 1
    
    return pd.Series(ret)

In [ ]:
# for the single class usage, i have only images in the test_path2 and i put manually the label to each
def getSingleLabelList(path):
    ret = {}
    index = 0
    for img in [ name for name in os.listdir(path) ]:
        ret[index] = [img, LABEL_INDEX]
        index += 1
    
    return pd.Series(ret)

In [ ]:
# create the pool of images (with a series that contains all the images, more deeply: index -> abs path, label (int), class (string))

def getImgPool(path):
    only_dirs = [ name for name in os.listdir(path) if 
                 os.path.isdir(os.path.join(path, name)) ]

    ret = {}
    index = 0
    
    for d in only_dirs:
        new_path = path + d
        label = only_dirs.index(d)
        # cnt = 0
        for img in [ name for name in os.listdir(new_path) ]:
            abs_path = new_path + '/' + img
            ret[index] = [abs_path, label, d]
            index += 1
            # cnt += 1
        # print(f"Label: {d} - Number: {cnt}")
    
    return pd.Series(ret)

In [ ]:
# create the pool of images from video (with a series that contains all the images, more deeply: index -> abs path, label (int), class (string))

def getSingleImgPool(path):
    ret = {}
    index = 0
    for img in [ name for name in os.listdir(path) ]:
        abs_path = path + '/' + img
        ret[index] = [abs_path, LABEL_INDEX, SINGLE_CLASS]
        index += 1
    
    return pd.Series(ret)

### Defining the custom dataset (structure that holds the data)

#### old version

In [ ]:
if not using_res:
    # wrong_images_path = []
    # import traceback

    class CustomImageDataset(Dataset):
        def __init__(self, path_labels, transform=None, target_transform=None, use_aug=True, single_class=None):
            if single_class is not None:
                self.img_labels = getSingleLabelList(path_labels)
                self.images = getSingleImgPool(path_labels)
            else:
                self.img_labels = getLabelList(path_labels)
                self.images = getImgPool(path_labels)
            self.transform = transform
            self.target_transform = target_transform
            self.use_aug = use_aug

        def __len__(self):
            return len(self.img_labels)

        def __getitem__(self, idx):
            try: 
                image = read_image(self.images.loc[idx][0])
            except Exception:
                print('Found error at {} {}'.format(self.images.loc[idx][0], idx))
            #     wrong_images_path.append(self.images.loc[img_name][0])
                #return torch.zeros((1, 600, 600)), -1
            # image = read_image(self.images.loc[img_name][0])
            if image.shape[0] >= 3: #apply grayscale to colored img
                image = to_tensor(to_grayscale(ToPILImage()(image), num_output_channels=1))
            else: #we have a greyscaled image, move it into tensor 
                image = to_tensor(ToPILImage()(image))

            image = Resize((300, 300))(image)
            #data augmentation
            if self.use_aug:
                fr = random.choices([0,1],[0.2,0.8])[0]
                if fr == 1: 
                    angle = torch.randint(40, size=(1,)).item() - 20
                    #print (f'Rotate: {angle}')
                    image = rotate(image, angle)

                fr = random.choices([0,1],[0.5,0.5])[0]
                if fr == 1: 
                    image = hflip(image)

            #image = image.permute(1,2,0) #made to be shown on matplotlib
            label = self.img_labels.loc[idx][1]
            #if self.transform:
            #    image = self.transform(image)
            #if self.target_transform:
            #    label = self.target_transform(label)

            return image, label

#### new version (resnet)

In [ ]:
if using_res:
    wrong_images_path = []

    class CustomImageDataset(Dataset):
        def __init__(self, path_labels, transform=None, target_transform=None, use_aug=True, single_class=None):
            if single_class is not None:
                self.img_labels = getSingleLabelList(path_labels)
                self.images = getSingleImgPool(path_labels)
            else:
                self.img_labels = getLabelList(path_labels)
                self.images = getImgPool(path_labels)
            self.transform = transform
            self.target_transform = target_transform
            self.use_aug = use_aug

        def __len__(self):
            return len(self.img_labels)

        def __getitem__(self, idx):
            try: 
                image = Image.open(self.images.loc[idx][0])
                # exif_data = image._getexif()
                if Image.MIME[image.format] == 'image/png': #wrong format, we want only jpg
                        image = convertPNGImage(self.images.loc[idx][0], 'PNG')
            # except UserWarning:
            #     print('Found exif error at {} in position {}'.format(self.images.loc[idx][0], idx))
            #     wrong_images_path.append('EXIF '+self.images.loc[idx][0])
            except Exception:
                print('Found exif error at {} in position {}'.format(self.images.loc[idx][0], idx))
                # wrong_images_path.append(self.images.loc[idx][0])
                
            image = ToTensor()(image)
            
            if image.shape[0] != 3: #color grayscaled and convert RGBA
                image = ToPILImage()(image).convert('RGB')
                
            else: #rgb, don't touch it 
                image = ToPILImage()(image)
                
            preprocess = Compose([
                Resize(300),
                CenterCrop(300),
                ToTensor(),
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image = preprocess(image)
            
            
            #  COMMENT ME
            # print(f"idx: {idx} - path: {self.images.loc[idx][0]}")
            
            
            
            #data augmentation
            if self.use_aug:
                fr = random.choices([0,1],[0.2,0.8])[0]
                if fr == 1: 
                    angle = torch.randint(40, size=(1,)).item() - 20
                    #print (f'Rotate: {angle}')
                    image = rotate(image, angle)

                fr = random.choices([0,1],[0.5,0.5])[0]
                if fr == 1: 
                    image = hflip(image)

            label = self.img_labels.loc[idx][1]

            return image, label

### Creating the 2 dataset and testing

In [ ]:
# train_data = CustomImageDataset(path_labels=train_path, transform=ToTensor(), num_of_channels=3)
# test_data = CustomImageDataset(path_labels=test_path, transform=ToTensor(), num_of_channels=3)

test_data = CustomImageDataset(path_labels=test_path2, transform=ToTensor(), 
                               target_transform=ToTensor(), use_aug=False, single_class=SINGLE_CLASS)

print (f'test dataset: {test_data.__len__()}')

In [ ]:
#testing dataset if images are loaded

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(test_data), size=(1,)).item() 
    img, label = test_data[sample_idx]
    if using_res:
        img = img * torch.tensor([0.229, 0.224, 0.225]).reshape(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).reshape(3,1,1)
    # print (img.shape, img.dtype, label)
    figure.add_subplot(rows, cols, i)
    plt.title(diz[label])
    plt.axis("off")
    if not using_res:
        plt.imshow(img.squeeze(), cmap='gray')
    else:
        plt.imshow(img.permute(1,2,0)) #use this if rbg
plt.show()

### creating dataloader (who iterate on data, create batches and shuffles it)


In [ ]:
test_dl = DataLoader(test_data, batch_size=24, shuffle=True)

#dataloader test (ask a batch or a sample)
images, labels = next(iter(test_dl)) #picking a batch
print(f"Feature batch shape: {images.size()}")
print(f"Labels batch shape: {labels.size()}")
img = images[0] #picking the first img and label of the batch

if using_res:
    img = img * torch.tensor([0.229, 0.224, 0.225]).reshape(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).reshape(3,1,1)
label = labels[0]
if not using_res:
    plt.imshow(img.squeeze(), cmap='gray')
else:
    plt.imshow(img.permute(1,2,0)) #use this if rbg
plt.show()
print(f"Label: {diz[label.item()]}")

In [ ]:
labels

### Defining the neural network

In [ ]:
print("GPU compute available: ", torch.cuda.is_available())


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
if not using_res:
    class Net(nn.Module):

        def __init__(self):
            super(Net, self).__init__()

            #conv1: 1 input image channel (image channel, 1 gray, 3 rgb), 6 output channels (depth [K]), 5x5 square convolution kernel, DEFAULT: stride = 1,1, padding = 0
            self.conv1 = nn.Conv2d(1, 6, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(6, 16, 5)        
            self.fc_layer1 = nn.Linear(16 * 72 * 72, 120)        
            self.fc_layer2 = nn.Linear(120, 84)
            self.fc_layer3 = nn.Linear(84, nocl) # nocl is the num of classes

        def forward(self, x):
            # Max pooling over a (2, 2) window
            x = self.pool(F.relu(self.conv1(x)))
            #print ('conv1', x.shape)
            # If the size is a square, you can specify with a single number
            x = self.pool(F.relu(self.conv2(x)))
            x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
            #print ('flatten', x.shape)
            x = F.relu(self.fc_layer1(x))
            x = F.relu(self.fc_layer2(x))
            x = self.fc_layer3(x)
            return x

### Testing the network on test set

In [ ]:
if not using_res:
    loaded = Net()
else:
    loaded = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

In [ ]:
loaded.load_state_dict(torch.load(SAVE_PATH))
loaded.eval()

In [ ]:
test_dl = DataLoader(test_data, batch_size=24)
images, labels = next(iter(test_dl)) #picking a batch

n_imgs = len(images)

preds = loaded(images)
_, best_pred = torch.max(preds.data, 1)

figure = plt.figure(figsize=(24, 24))
cols, rows = 4, int(n_imgs/4)
for i in range(1, cols * rows + 1):
    figure.add_subplot(rows, cols, i)
    plt.title('T: {}\nP: {}'.format(diz[labels[i-1].item()], diz[best_pred[i-1].item()]))
    plt.axis("off")
    if not using_res:
        plt.imshow(images[i-1].squeeze(), cmap='gray')
    else:
        images[i-1] = images[i-1] * torch.tensor([0.229, 0.224, 0.225]).reshape(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).reshape(3,1,1)
        plt.imshow(images[i-1].permute(1,2,0)) #use this if rbg
plt.show()

# axs[i].set_title('T: {}\nP: {}'.format(diz[label[i].item()], diz[best_pred[i].item()]))

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in diz}
total_pred = {classname: 0 for classname in diz}

y_true = []
y_pred = []

# again no gradients needed
# with torch.no_grad():
for data in test_dl:
    images, labels = data
    outputs = loaded(images)
    _, predictions = torch.max(outputs, 1)
    # collect the correct predictions for each class
    for label, prediction in zip(labels, predictions):
        if label == prediction:
            correct_pred[label.item()] += 1
        else:
            correct_pred[prediction.item()] += 1 
        total_pred[label.item()] += 1
        y_true.append(label.cpu())
        y_pred.append(prediction.cpu())

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    if total_pred[classname] == 0: #wrong class pred
        accuracy = 100 * float(correct_count) / test_data.__len__()
    else:
        accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Predictions for class: {diz[classname]} is {accuracy:.1f} %')